In [20]:
%matplotlib inline
#%load_ext autoreload
#%autoreload 2

import keras
import keras.preprocessing.image
from keras_retinanet.models.resnet import custom_objects
from keras_retinanet.preprocessing.coco import CocoGenerator
from keras_retinanet.preprocessing.csv_generator import CSVGenerator

import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
import time

import tensorflow as tf

def get_session():
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    return tf.Session(config=config)

# os.environ["CUDA_VISIBLE_DEVICES"] = "0"
keras.backend.tensorflow_backend.set_session(get_session())

In [21]:
def get_area(points):
    area = (int(points[2]) - int(points[0])) * (int(points[3]) - int(points[1]))
    return area

def getIoU(answer, result):
    def get_inter_area(answer, result):
        inter_x1 = max(int(answer[0]), int(result[0]))
        inter_y1 = max(int(answer[1]), int(result[1]))
        inter_x2 = min(int(answer[2]), int(result[2]))
        inter_y2 = min(int(answer[3]), int(result[3]))
        if (inter_x2 - inter_x1) >= 0 and (inter_y2 - inter_y1) >= 0:
            inter = [inter_x1, inter_y1, inter_x2, inter_y2]
            inter_area = get_area(inter)
            return inter_area
        else:
            return 0
    
        
    answer_area = get_area(answer)
    result_area = get_area(result)
    inter_area = get_inter_area(answer, result)
    #print answer
    #print result
    #print answer_area, result_area, inter_area
    if inter_area != 0:
        IoU = max(float(inter_area) / float(answer_area + result_area - inter_area), 0.)
    else:
        IoU = 0.
    return IoU

def isCover(answer, result):
    iscovered = False
    if int(answer[0]) <= int(result[0]):
        if int(answer[1]) <= int(result[1]):
            if int(answer[2]) >= int(result[2]):
                if int(answer[3]) >= int(result[3]):
                    iscovered = True
    if int(result[0]) <= int(answer[0]):
        if int(result[1]) <= int(answer[1]):
            if int(result[2]) >= int(answer[2]):
                if int(result[3]) >= int(answer[3]):
                    iscovered = True              
    return iscovered   

def point_adjust(point, img):
    point[0] = max(point[0], 0)
    point[1] = max(point[1], 0)
    point[2] = min(point[2], img.shape[1])
    point[3] = min(point[3], img.shape[0])    
    return point

def img_crop(img, count=3):
    
    height, width = img.shape[:2]
    crop_height = height / (count + 1)
    crop_width = width / (count + 1)

    crop_image_list = []
    for h_idx in range(count):
        for w_idx in range(count):
            if h_idx == (count - 1) and w_idx == (count - 1):
                start_y = h_idx * crop_height
                start_x = w_idx * crop_width
                end_x = width
                end_y = height

            elif h_idx != (count - 1) and w_idx == (count - 1):
                start_y = h_idx * crop_height
                start_x = w_idx * crop_width
                end_x = width
                end_y = (h_idx + 2) * crop_height     

            elif h_idx == (count - 1) and w_idx != (count - 1):
                start_y = h_idx * crop_height
                start_x = w_idx * crop_width
                end_x = (w_idx + 2) * crop_width
                end_y = height

            else:
                start_y = h_idx * crop_height
                start_x = w_idx * crop_width
                end_x = (w_idx + 2) * crop_width
                end_y = (h_idx + 2) * crop_height   
            crop_img = img[start_y:end_y, start_x:end_x, :].copy()
            crop_image_list.append((crop_img, start_x, start_y)) 
    return crop_image_list
def mergeAnnotation(annotation_list):
    
    def iteration(annotation_list):
        total_set_list = []
        merged_list = []
        for annotation in annotation_list:
            if annotation[1] > 0.1:
                if len(total_set_list) == 0:
                    total_set_list.append([annotation])
                else:
                    isAppended = False
                    for sets in total_set_list:

                        for one in sets:
                            IoU = getIoU(one[0], annotation[0])
                            isCovered = isCover(one[0], annotation[0])
                            #if IoU > 0.2 or isCovered == True:
                            if IoU > 0.2:
                                sets.append(annotation)
                                isAppended = True
                                break

                        if isAppended == True:
                            break

                    if isAppended == False:
                        total_set_list.append([annotation])    


        for sets in total_set_list:
            value = 0
            max_area = 0
            max_info = None
            max_value = 0
            for idx, one in enumerate(sets):
                area = get_area(one[0])
                value += one[1]

                if area > max_area:
                    max_area = area
                    max_info = one
                if float(one[1]) > max_value:
                    max_value = float(one[1])

            value = float(value) / len(sets)
            merged_list.append((max_info[0], max_value))
        return merged_list
    
    
    final_list = iteration(annotation_list)
    length = len(final_list)
    for i in range(100):
        final_list = iteration(final_list)
        if length == len(final_list):
            break
    return final_list

In [23]:
def run(indexx, cv, data_type, visualize=True):

    model_name = data_type + '_' + str(indexx) + '_' + str(cv) + '_train_0.h5'
    model_path = os.path.join('/media/hwejin/SSD_1/Code/Github/keras-retinanet/snapshots', model_name)
    model = keras.models.load_model(model_path, custom_objects=custom_objects)
    
    
    
    
    
    sets = ['_test']
    for setss in sets:
        test_name = str(indexx) + '_' + str(cv) + setss
        #test_name = str(indexx) + setss
        
        test_path = os.path.join('/media/hwejin/SSD_1/DATA/DATAForPaper_v2/Annotation/FINAL', data_type, test_name + '.csv')
        class_path = '/media/hwejin/SSD_1/DATA/DATAForPaper_v2/Common/class.csv'


        out_dir_path = os.path.join('/media/hwejin/SSD_1/DATA/DATAForPaper_v2/Result/', data_type)
        if not os.path.isdir(out_dir_path):
            os.mkdir(out_dir_path)
        out_dir_path = os.path.join(out_dir_path, test_name)
        if not os.path.isdir(out_dir_path):
            os.mkdir(out_dir_path)



        # create image data generator object
        val_image_data_generator = keras.preprocessing.image.ImageDataGenerator()

        # create a generator for testing data
        val_generator = CSVGenerator(
            test_path,
            class_path,
            '-',
            val_image_data_generator,
            batch_size=1)



        
        for index in range(len(val_generator.image_data)):
            total_annotation_list = []
            # load image
            Load_image = val_generator.load_image(index)
            # copy to draw on
            draw = Load_image.copy()
            draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)
            

            
            annotations = val_generator.load_annotations(index)
            
            
            crop_image_list = img_crop(Load_image, count=5)
            crop_image_list.append((Load_image, 0, 0))

            
            
            
            
            for crop_image_idx, image_info in enumerate(crop_image_list):
                image = image_info[0]
                slice_img = image.copy()
                start_x = image_info[1]
                start_y = image_info[2]
            
                # preprocess image for network
                image = val_generator.preprocess_image(image)
                image, scale = val_generator.resize_image(image)


                # process image
                start = time.time()
                _, _, detections = model.predict_on_batch(np.expand_dims(image, axis=0))
                #print("processing time: ", time.time() - start)

                # compute predicted labels and scores
                predicted_labels = np.argmax(detections[0, :, 4:], axis=1)
                scores = detections[0, np.arange(detections.shape[1]), 4 + predicted_labels]


                #print val_generator.image_names[index]

                # correct for image scale
                detections[0, :, :4] /= scale


                # visualize detections
                for idx, (label, score) in enumerate(zip(predicted_labels, scores)):

                    b = detections[0, idx, :4].astype(int)
                    b = point_adjust(b, draw)
                    
                    
                    b[0] = b[0] + start_x
                    b[1] = b[1] + start_y
                    b[2] = b[2] + start_x
                    b[3] = b[3] + start_y
                    total_annotation_list.append((b, score))
                    
                
                
                


            merged_list = mergeAnnotation(total_annotation_list)  
            for annotation in merged_list:
                if annotation[1] > 0.1:
                    area = get_area(annotation[0])
                    caption = str(round(annotation[1], 3))#"{} {:.3f}".format(val_generator.label_to_name(label), score)
                    
                    if area > 5000:
                        cv2.rectangle(draw, (annotation[0][0], annotation[0][1]), (annotation[0][2], annotation[0][3]), (255, 0, 0), 5)
                        cv2.putText(draw, caption, (annotation[0][0], annotation[0][1] - 10), cv2.FONT_HERSHEY_PLAIN, 3, (255, 0, 0), 5)
                    #else:
                        #cv2.rectangle(draw, (annotation[0][0], annotation[0][1]), (annotation[0][2], annotation[0][3]), (255, 255, 0), 5)
                    #cv2.putText(draw, caption, (annotation[0][0], annotation[0][1] - 10), cv2.FONT_HERSHEY_PLAIN, 3, (255, 0, 0), 5)
                    #caption = str(get_area(annotation[0]))
                    #cv2.putText(draw, caption, (annotation[0][0], annotation[0][3] + 40), cv2.FONT_HERSHEY_PLAIN, 3, (255, 0, 0), 3)
            # visualize annotations
            for annotation in annotations:
                label = int(annotation[4])
                b = annotation[:4].astype(int)
                cv2.rectangle(draw, (b[0], b[1]), (b[2], b[3]), (0, 255, 0), 5)
                #caption = "{}".format(val_generator.label_to_name(label))
                #cv2.putText(draw, caption, (b[0], b[1] - 10), cv2.FONT_HERSHEY_PLAIN, 2.5, (0, 255, 0), 2)

                
            if visualize:
                print val_generator.image_names[index]
                plt.figure(figsize=(20, 20))
                plt.axis('off')
                plt.imshow(draw)
                plt.show()
                out_img_path = os.path.join('/media/hwejin/SSD_1/DATA/DATAForPaper_v2/OUT/', val_generator.image_names[index])
                
                cv2.imwrite(out_img_path, draw)
                
            
            
            
          
        
            ########total_annotation_list  -> merge
        
            f = open(os.path.join(out_dir_path, val_generator.image_names[index].replace('png', 'txt')), 'w')
            
            for annotation in merged_list:
                f.write(str(annotation[0][0]) + ',' + str(annotation[0][1]) + ',' + str(annotation[0][2]) + ',' + str(annotation[0][3]) + ',' + str(annotation[1]) + '\n')
            f.close()           

In [26]:

sampling_count = 0
dataset = ['MIX', 'GI-Iv2', 'INBreast', 'GURO', 'GI-Gv2']
dataset = ['GURO', 'INBreast']
#dataset = ['INBreast']
for data in dataset:
    for i in range(sampling_count, sampling_count + 1):
        for cv in range(5):
            print data, cv
            run(i, cv, data, visualize=False)
            print '-----------------------------------------------------------------------------------------'
    print data, 'DONE'

GURO 0
-----------------------------------------------------------------------------------------
GURO 1
-----------------------------------------------------------------------------------------
GURO 2
-----------------------------------------------------------------------------------------
GURO 3
-----------------------------------------------------------------------------------------
GURO 4
-----------------------------------------------------------------------------------------
GURO DONE
INBreast 0
-----------------------------------------------------------------------------------------
INBreast 1
-----------------------------------------------------------------------------------------
INBreast 2
-----------------------------------------------------------------------------------------
INBreast 3
-----------------------------------------------------------------------------------------
INBreast 4
-----------------------------------------------------------------------------------------
